<a href="https://colab.research.google.com/github/ParkSeonungHun/Hun/blob/main/python_basic/%EC%BB%AC%EB%9F%AC_%EB%A7%B5_%EC%A0%95%EA%B7%9C%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 컬러 맵 정규화

기본적으로 컬러 맵을 사용하는 객체는 데이터 값 vmin 에서 vmax로 컬러 맵의 색상을 선형으로 매핑합니다 . 예를 들면 :

In [ ]:
pcm = ax.pcolormesh(x, y, Z, vmin=-1., vmax=1., cmap='RdBu_r')

Z 의 데이터 를 -1에서 +1까지 선형으로 매핑 하므로 Z = 0 은 컬러 맵 RdBu_r (이 경우 흰색) 의 중심에 색상을 제공합니다 .

Matplotlib는 입력 데이터에서 [0, 1] 로의 정규화를 먼저 수행 한 다음 컬러 맵의 인덱스에 매핑하는 두 단계로이 매핑을 수행합니다. 정규화는 matplotlib.colors()모듈에 정의 된 클래스 입니다. 기본 선형 정규화는 matplotlib.colors.Normalize()입니다.

데이터를 색상에 매핑하는 아티스트는 vmin 및 vmax 인수를 전달하여 matplotlib.colors.Normalize()인스턴스 를 생성 한 다음이를 호출합니다.

In [1]:
In [1]: import matplotlib as mpl

In [2]: norm = mpl.colors.Normalize(vmin=-1, vmax=1)

In [3]: norm(0)
Out[3]: 0.5

그러나 때때로 비선형 방식으로 데이터를 컬러 맵에 매핑하는 것이 유용한 경우가 있습니다.

## 로그

가장 일반적인 변환 중 하나는 로그 (밑수 10)를 취하여 데이터를 플로팅하는 것입니다. 이 변환은 서로 다른 배율에 걸쳐 변경 사항을 표시하는 데 유용합니다. 사용 colors.LogNorm은 다음을 통해 데이터를 정규화합니다. log10. 아래 예에는 두 개의 범프가 있으며 하나는 다른 하나보다 훨씬 작습니다. 를 사용하면 colors.LogNorm각 범프의 모양과 위치를 명확하게 볼 수 있습니다.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook


In [ ]:
N = 100
X, Y = np.mgrid[-3:3:complex(0, N), -2:2:complex(0, N)]

# A low hump with a spike coming out of the top right.  Needs to have
# z/colour axis on a log scale so we see both hump and spike.  linear
# scale only shows the spike.
Z1 = np.exp(-X**2 - Y**2)
Z2 = np.exp(-(X * 10)**2 - (Y * 10)**2)
Z = Z1 + 50 * Z2

fig, ax = plt.subplots(2, 1)

pcm = ax[0].pcolor(X, Y, Z,
                   norm=colors.LogNorm(vmin=Z.min(), vmax=Z.max()),
                   cmap='PuBu_r', shading='auto')
fig.colorbar(pcm, ax=ax[0], extend='max')

pcm = ax[1].pcolor(X, Y, Z, cmap='PuBu_r', shading='auto')
fig.colorbar(pcm, ax=ax[1], extend='max')
plt.show()

## 대칭 로그

마찬가지로 양수와 음수 데이터가있을 때도 있지만 두 가지 모두에 대수 스케일링을 적용하고 싶습니다. 이 경우 음수도 대수적으로 스케일링되고 더 작은 숫자로 매핑됩니다. 예를 들어이면 vmin=-vmax음수는 0에서 0.5로, 양수는 0.5에서 1로 매핑됩니다.

0에 가까운 값의 로그는 무한대에 가까워 지므로 0 주변의 작은 범위를 선형으로 매핑해야합니다. linthresh 매개 변수를 사용하면 사용자가이 범위의 크기를 지정할 수 있습니다 ( -linthresh , linthresh ). 컬러 맵에서이 범위의 크기는 linscale에 의해 설정됩니다 . 경우 linscale == 1.0 (기본값), 선형 범위의 양 및 음의 절반에 사용되는 공간 대수 범위 일 년간 동일 할 것이다.

In [ ]:
N = 100
X, Y = np.mgrid[-3:3:complex(0, N), -2:2:complex(0, N)]
Z1 = np.exp(-X**2 - Y**2)
Z2 = np.exp(-(X - 1)**2 - (Y - 1)**2)
Z = (Z1 - Z2) * 2

fig, ax = plt.subplots(2, 1)

pcm = ax[0].pcolormesh(X, Y, Z,
                       norm=colors.SymLogNorm(linthresh=0.03, linscale=0.03,
                                              vmin=-1.0, vmax=1.0, base=10),
                       cmap='RdBu_r', shading='auto')
fig.colorbar(pcm, ax=ax[0], extend='both')

pcm = ax[1].pcolormesh(X, Y, Z, cmap='RdBu_r', vmin=-np.max(Z), shading='auto')
fig.colorbar(pcm, ax=ax[1], extend='both')
plt.show()

## 법칙

때로는 색상을 멱 법칙 관계에 다시 매핑하는 것이 유용합니다 (예 : y=xγ, 어디 γ힘입니다). 이를 위해 우리는 colors.PowerNorm. 감마 인수로 사용합니다 ( 감마 == 1.0은 기본 선형 정규화를 생성합니다).

**노트**

이러한 유형의 변환을 사용하여 데이터를 플로팅하는 데에는 좋은 이유가있을 것입니다. 기술 뷰어는 선형 및 로그 축과 데이터 변환에 사용됩니다. 권력 법칙은 덜 일반적이며 시청자는 그들이 사용되었음을 명시 적으로 인식해야합니다.

In [ ]:
N = 100
X, Y = np.mgrid[0:3:complex(0, N), 0:2:complex(0, N)]
Z1 = (1 + np.sin(Y * 10.)) * X**2

fig, ax = plt.subplots(2, 1)

pcm = ax[0].pcolormesh(X, Y, Z1, norm=colors.PowerNorm(gamma=0.5),
                       cmap='PuBu_r', shading='auto')
fig.colorbar(pcm, ax=ax[0], extend='max')

pcm = ax[1].pcolormesh(X, Y, Z1, cmap='PuBu_r', shading='auto')
fig.colorbar(pcm, ax=ax[1], extend='max')
plt.show()

## 이산 경계

Matplotlib와 함께 제공되는 또 다른 정규화는 colors.BoundaryNorm. vmin 및 vmax 외에도 데이터를 매핑 할 인수 경계로 사용합니다. 그런 다음 색상이 이러한 "경계"간에 선형으로 분포됩니다. 또한 extend 인수를 사용하여 색상이 분산되는 범위에 상한 및 / 또는 하한 범위를 벗어난 값을 추가 할 수 있습니다 . 예를 들면 :

In [ ]:
In [4]: import matplotlib.colors as colors

In [5]: bounds = np.array([-0.25, -0.125, 0, 0.5, 1])

In [6]: norm = colors.BoundaryNorm(boundaries=bounds, ncolors=4)

In [7]: print(norm([-0.2, -0.15, -0.02, 0.3, 0.8, 0.99]))
[0 0 1 2 3 3]

참고 : 다른 표준과 달리이 표준은 0에서 ncolors -1 까지의 값을 반환합니다 .

In [ ]:
N = 100
X, Y = np.meshgrid(np.linspace(-3, 3, N), np.linspace(-2, 2, N))
Z1 = np.exp(-X**2 - Y**2)
Z2 = np.exp(-(X - 1)**2 - (Y - 1)**2)
Z = ((Z1 - Z2) * 2)[:-1, :-1]

fig, ax = plt.subplots(2, 2, figsize=(8, 6), constrained_layout=True)
ax = ax.flatten()

# Default norm:
pcm = ax[0].pcolormesh(X, Y, Z, cmap='RdBu_r')
fig.colorbar(pcm, ax=ax[0], orientation='vertical')
ax[0].set_title('Default norm')

# Even bounds give a contour-like effect:
bounds = np.linspace(-1.5, 1.5, 7)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
pcm = ax[1].pcolormesh(X, Y, Z, norm=norm, cmap='RdBu_r')
fig.colorbar(pcm, ax=ax[1], extend='both', orientation='vertical')
ax[1].set_title('BoundaryNorm: 7 boundaries')

# Bounds may be unevenly spaced:
bounds = np.array([-0.2, -0.1, 0, 0.5, 1])
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
pcm = ax[2].pcolormesh(X, Y, Z, norm=norm, cmap='RdBu_r')
fig.colorbar(pcm, ax=ax[2], extend='both', orientation='vertical')
ax[2].set_title('BoundaryNorm: nonuniform')

# With out-of-bounds colors:
bounds = np.linspace(-1.5, 1.5, 7)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256, extend='both')
pcm = ax[3].pcolormesh(X, Y, Z, norm=norm, cmap='RdBu_r')
# The colorbar inherits the "extend" argument from BoundaryNorm.
fig.colorbar(pcm, ax=ax[3], orientation='vertical')
ax[3].set_title('BoundaryNorm: extend="both"')
plt.show()

## TwoSlopeNorm : 중심의 양쪽에 다른 매핑

때때로 우리는 개념적 중심점의 양쪽에 다른 컬러 맵을 갖고 싶고,이 두 컬러 맵이 다른 선형 스케일을 갖기를 원합니다. 예를 들어 육지와 바다의 중심이 0에 있지만 일반적으로 육지는 수심 범위보다 해발 범위가 더 넓고 종종 다른 컬러 맵으로 표시되는 지형도를들 수 있습니다.

In [ ]:
dem = cbook.get_sample_data('topobathy.npz', np_load=True)
topo = dem['topo']
longitude = dem['longitude']
latitude = dem['latitude']

fig, ax = plt.subplots()
# make a colormap that has land and ocean clearly delineated and of the
# same length (256 + 256)
colors_undersea = plt.cm.terrain(np.linspace(0, 0.17, 256))
colors_land = plt.cm.terrain(np.linspace(0.25, 1, 256))
all_colors = np.vstack((colors_undersea, colors_land))
terrain_map = colors.LinearSegmentedColormap.from_list(
    'terrain_map', all_colors)

# make the norm:  Note the center is offset so that the land has more
# dynamic range:
divnorm = colors.TwoSlopeNorm(vmin=-500., vcenter=0, vmax=4000)

pcm = ax.pcolormesh(longitude, latitude, topo, rasterized=True, norm=divnorm,
                    cmap=terrain_map, shading='auto')
# Simple geographic plot, set aspect ratio beecause distance between lines of
# longitude depends on latitude.
ax.set_aspect(1 / np.cos(np.deg2rad(49)))
fig.colorbar(pcm, shrink=0.6)
plt.show()

## 사용자 지정 정규화 : 두 개의 선형 범위를 수동으로 구현

는 TwoSlopeNorm위에서 설명한 자신의 표준을 정의하기위한 유용한 예를 만든다.

In [ ]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))


fig, ax = plt.subplots()
midnorm = MidpointNormalize(vmin=-500., vcenter=0, vmax=4000)

pcm = ax.pcolormesh(longitude, latitude, topo, rasterized=True, norm=midnorm,
                    cmap=terrain_map, shading='auto')
ax.set_aspect(1 / np.cos(np.deg2rad(49)))
fig.colorbar(pcm, shrink=0.6, extend='both')
plt.show()